## Files

In [27]:
import os 
import pandas as pd
import json
import ast
import re
import json
from tqdm import tqdm
import lorem

foundation_model = "openai"
# foundation_model = "gemini"

# read four dirs email.csv
# weeks = range(1, 5)
weeks = range(1, 4)

scenario = "Tech-Company"
company_type_member = "tech_company"
company_type = "tech"

# scenario = "Finance-Company"
# company_type_member = "finance_corporation"
# company_type = "finance"

# scenario = "Medical-Institution"
# company_type_member = "medical_institution"
# company_type = "medical"

# Reading
log_root_dir = f'/data2/visitor/ASE25/Chimera-Dataset/'
root_dir = "/data2/visitor/ASE25/Chimera"

# output_dir = f"{root_dir}/Final-Output/{company_type_member}/"
output_dir = f"{root_dir}/Final-Output-OpenAI/{company_type_member}/"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

output_csv = f'{output_dir}/{company_type_member}_file_raw.csv'

id_role_map = {}
id_list = []
profile_list = []

#######################
profile_output_dir = f"{root_dir}/experiment_output/{foundation_model}_{company_type_member}/generated_members"
for file in os.listdir(profile_output_dir):
    if file.endswith(".jsonc"):
        member_profile_path = os.path.join(profile_output_dir, file)
        with open(member_profile_path, 'r') as f:
            member_profile = json.load(f)
        id_role_map[member_profile['id']] = member_profile['role'] # add id-role map
        id_list.append(member_profile['id'])
        profile_list.append(member_profile) # add profile
#######################

data_rows = []
last_url = None

last_time = None

for week in weeks:
    weekly_dir = os.path.join(log_root_dir, scenario, f'week{week}-{foundation_model}-{company_type}')
    execution_log_path = os.path.join(weekly_dir, "execution_logs")
    
    for member_id in tqdm(id_list, desc=f"Processing member logs for week {week}", leave=False):    
        user_log_dir = os.path.join(execution_log_path, member_id)

        # for every file inside the user_log_dir, read the file and extract the content and do not read the file with "solution"
        log_files = [f for f in os.listdir(user_log_dir) if f.endswith('.log') and 'solution' not in f]
        for member_action_log in tqdm(log_files, desc=f"Processing logs for {member_id}", leave=False):
            member_action_log_path = os.path.join(user_log_dir, member_action_log)
            
            with open(member_action_log_path, 'r') as f:
                content = f.readlines()
            
            for line in content:
                
                if "'tool_calls': [{'id': 'null', 'type': 'function', 'function': {'name':" in line:

                    match = re.search(r"'name': '([^']+)'", line)
                    time = line.split(',')[0].strip()
                    if match:
                        function_name = match.group(1)
                        # 'tool_calls': [{'id': 'null', 'type':
                        # {'shell_exec', 'browse_url', 'search_duckduckgo', 'file_find_in_content', 'write_to_file', 'search_google', 'file_find_by_name'}
                        if function_name == 'shell_exec':
                            match = re.search(r'"command"\s*:\s*"([^"]+)"', line)
                            if match:
                                command_data = match.group(1)
                                # pass

                        elif function_name == 'file_find_in_content':
                            match = re.search(r"'arguments':\s*'\{\"file\":\s*\"([^\"]+)\"", line)
                            if match:
                                file_read = match.group(1)
                                row = {
                                    'real_timestamp': time,
                                    'id': member_id,
                                    'filename': file_read,
                                    'type': 'read',
                                    'content': lorem.sentence()
                                }
                            data_rows.append(row)
                        
                        elif function_name == 'file_find_by_name':
                            matches = re.findall(r"'arguments':\s*'\{\"glob\":\s*\"([^\"]+)\"", line)
                            for match in matches:
                                file_name = match
                                row = {
                                    'real_timestamp': time,
                                    'id': member_id,
                                    'filename': file_name,
                                    'type': 'read',
                                    'content': lorem.sentence()
                                }
                                data_rows.append(row)
                        
                        elif function_name == 'write_to_file':
                            content_match = re.search(r"'arguments':\s*'\{\"content\":\s*\"([^\"]+)\"", line)
                            filename_match = re.search(r'"filename"\s*:\s*"([^"]+)"', line)

                            row = {
                                'real_timestamp': time,
                                'id': member_id,
                                'filename': filename_match.group(1),
                                'type': 'write',
                                'content': content_match.group(1) if content_match else lorem.sentence()
                            }
                            data_rows.append(row)
                
                if "Content successfully written to file" in line:
                    time = line.split(',')[0].strip()
                    match = re.search(r"'content': 'Content successfully written to file: ([^']+)'", line)
                    if match:
                        file_written = match.group(1)
                        row = {
                            'real_timestamp': time,
                            'id': member_id,
                            'filename': file_written,
                            'type': 'write',
                            'content': lorem.sentence()
                        }
                        data_rows.append(row)

df = pd.DataFrame(data_rows)

df.to_csv(output_csv, index=False, encoding='utf-8')

In [28]:
df.head(5)

,real_timestamp,id,filename,type,content
0,2025-05-16 15:49:55,lpro-1,/data/Chimera/demo/execution_logs/lpro-1...,write,Modi amet tempora voluptatem etincidunt dolore...
1,2025-05-16 17:57:28,lpro-1,/data/Chimera/demo/execution_logs/lpro-1...,write,Est non numquam adipisci sit magnam quiquia ad...
2,2025-05-16 15:41:25,3dmod-2,/data/Chimera/demo/execution_logs/3dmod-...,write,Ipsum ut aliquam quisquam.
3,2025-05-16 15:41:16,3dmod-2,/data/Chimera/demo/execution_logs/3dmod-...,write,Amet ipsum porro numquam ut.
4,2025-05-16 15:23:22,3dmod-2,/data/Chimera/demo/execution_logs/3dmod-...,write,Non magnam est consectetur sed dolore modi.


### Process Raw

In [29]:
import random

# read four dirs email.csv
# weeks = range(1, 5)
weeks = range(1, 4)

# output_dir = f"{root_dir}/Final-Output/{company_type_member}/"
output_csv = f'{output_dir}/{company_type_member}_file.csv'

raw_csv = f'{output_dir}/{company_type_member}_file_raw.csv'

# reading the raw csv
raw_df = pd.read_csv(raw_csv, encoding='utf-8')

In [30]:
raw_df.head(5)

,real_timestamp,id,filename,type,content
0,2025-05-16 15:49:55,lpro-1,/data/Chimera/demo/execution_logs/lpro-1...,write,Modi amet tempora voluptatem etincidunt dolore...
1,2025-05-16 17:57:28,lpro-1,/data/Chimera/demo/execution_logs/lpro-1...,write,Est non numquam adipisci sit magnam quiquia ad...
2,2025-05-16 15:41:25,3dmod-2,/data/Chimera/demo/execution_logs/3dmod-...,write,Ipsum ut aliquam quisquam.
3,2025-05-16 15:41:16,3dmod-2,/data/Chimera/demo/execution_logs/3dmod-...,write,Amet ipsum porro numquam ut.
4,2025-05-16 15:23:22,3dmod-2,/data/Chimera/demo/execution_logs/3dmod-...,write,Non magnam est consectetur sed dolore modi.


In [31]:
# 1. match up with the actual time
logon_csv = f'{output_dir}/{company_type_member}_logon_full.csv'
logon_df = pd.read_csv(logon_csv, encoding='utf-8')

logon_df['real_timestamp'] = pd.to_datetime(logon_df['real_timestamp'])
logon_df['date'] = pd.to_datetime(logon_df['date'])
raw_df['real_timestamp'] = pd.to_datetime(raw_df['real_timestamp'])

raw_df = raw_df.sort_values('real_timestamp')
logon_df = logon_df.sort_values('real_timestamp')

result = pd.merge_asof(
    raw_df,
    logon_df,
    on='real_timestamp',
    by='id',
    direction='nearest'
)

# # 2. mutate the date time by up to 1 minute
def mutate_time(dt):
    # Randomly add or subtract up to 1 minute
    delta = pd.Timedelta(seconds=random.randint(-10, 10))
    return dt + delta
result['date'] = result['date'].apply(mutate_time)

# # 3. remove sim_timestamp, real_timestamp, activity
result.drop(columns=['sim_timestamp', 'real_timestamp', 'activity'], inplace=True)

# # 4. sort by date
result = result.sort_values(by='date', ascending=True).reset_index(drop=True)

In [32]:
result.head(5)

# 5. save to csv
result.to_csv(output_csv, index=False, encoding='utf-8')